In [118]:
import os
import pandas as pd
import numpy as np
import hvplot.pandas
from psaw import PushshiftAPI
from pathlib import Path
from newsapi import NewsApiClient
from dotenv import load_dotenv
from datetime import datetime
import datetime as dt
from dateutil.relativedelta import relativedelta, MO
import requests
import csv
from datetime import timezone
import time
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
load_dotenv()

True

In [120]:
api = PushshiftAPI()

In [119]:
# Download/Update the VADER Lexicon
nltk.download("vader_lexicon")

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dkv\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


**CARDANO - ZILLIQA - CHAINLINK**
--


**SCRAPE CARDANO SUBMISSIONS IN CRYPTOCURRENCY SUBREDDIT**

In [121]:
start_epoch=int(dt.datetime(2020, 5, 1).timestamp())

cardano_subm = api.search_submissions(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='cardano',
                            filter=['title', 'selftext'],
                            limit=None)


In [122]:
cardano_subm_df = pd.DataFrame([thing.d_ for thing in cardano_subm])

In [123]:
cardano_subm_df.tail()

,created,created_utc,selftext,title
129,1.588493e+09,1588478837,,Does Cardano need more respect?
130,1.588468e+09,1588453982,"Would EOS, Tezos , and especially CARDANO woul...",What positive attributes and technological lea...
131,1.588446e+09,1588432050,,Charles Hoskinson discusses the coronavirus re...
132,1.588411e+09,1588396528,,Cardano Risk Analysis
133,1.588365e+09,1588351046,,Comprehensive Cardano (ADA) Trading Guide : St...


In [124]:
cardano_subm_df.drop(['created'], axis=1, inplace=True)

In [125]:
cardano_subm_df['created_utc'] = pd.to_datetime(cardano_subm_df['created_utc'],unit='s')

In [126]:
cardano_subm_df.set_index(['created_utc'], inplace=True)

In [127]:
cardano_subm_df.head()

,selftext,title
created_utc,,
2020-07-11 14:12:40,,Seem like they feel the pressure from Cardano ...
2020-07-09 13:52:15,,Cardano Reaches Overbought Territory Despite H...
2020-07-09 13:30:12,,Cardano Foundation Warns About Suspicious Acti...
2020-07-09 11:48:07,"I was thinking of getting some XRP,Stellar,Ca...",Looking to get into altcoins for the first tim...
2020-07-08 22:04:47,,Cardano: ADA keeps climbing! But what is the A...


In [128]:
cardano_subm_df.shape

(134, 2)

In [129]:
cardano_subm_df.reset_index(drop=True)

,selftext,title
0,,Seem like they feel the pressure from Cardano ...
1,,Cardano Reaches Overbought Territory Despite H...
2,,Cardano Foundation Warns About Suspicious Acti...
3,"I was thinking of getting some XRP,Stellar,Ca...",Looking to get into altcoins for the first tim...
4,,Cardano: ADA keeps climbing! But what is the A...
5,[removed],Cardano is a joke
6,&amp;#x200B;\n\n[ EXPERIMENT - Tracking Top 10...,I bought $1k of the Top Ten Cryptos on January...
7,,3 Reasons Cardano (ADA) Suddenly Surged by 85%...
8,,Cardano (ADA) is now up more than 200% since t...
9,VET coming in first with a whopping 94% increa...,I bought $100 each of 10 PoS Currencies. 2 Wee...


In [131]:
#fill in null values with a placeholder
cardano_subm_df.fillna('.', inplace = True)
#create column 'body' that combines 'title' with 'selftext'
cardano_subm_df['cardano_body'] = cardano_subm_df['title'] + ' ' + cardano_subm_df['selftext']
#subset dataframe of Cardano posts
cardano_subm_df = cardano_subm_df[['cardano_body']]

cardano_subm_df.head()

,cardano_body
created_utc,
2020-07-11 14:12:40,Seem like they feel the pressure from Cardano ...
2020-07-09 13:52:15,Cardano Reaches Overbought Territory Despite H...
2020-07-09 13:30:12,Cardano Foundation Warns About Suspicious Acti...
2020-07-09 11:48:07,Looking to get into altcoins for the first tim...
2020-07-08 22:04:47,Cardano: ADA keeps climbing! But what is the A...


**SCRAPE CARDANO COMMENTS IN CRYPTOCURRENCY SUBREDDIT**

In [132]:
start_epoch=int(dt.datetime(2020, 5, 1).timestamp())

cardano_comm = api.search_comments(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='cardano',
                            filter=['body'],
                            limit=None)

In [133]:
cardano_comm_df = pd.DataFrame([thing.d_ for thing in cardano_comm])

C:\Users\dkv\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\dkv\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [134]:
cardano_comm_df.head()

,body,created,created_utc
0,cardano has research. at a fundamental level i...,1.594488e+09,1594473480
1,I recommend watching the Cardano Summit. In 20...,1.594483e+09,1594469051
2,Only Cardano is. And it pretty much is the Eth...,1.594482e+09,1594467856
3,"i have bags on both, but VeChain got some dapp...",1.594470e+09,1594455139
4,The reason Cardano 'has nothing' is precisely ...,1.594463e+09,1594449041


In [135]:
cardano_comm_df.drop(['created'], axis=1, inplace=True)

In [136]:
cardano_comm_df['created_utc'] = pd.to_datetime(cardano_comm_df['created_utc'],unit='s')

In [137]:
cardano_comm_df.set_index(['created_utc'], inplace=True)

In [138]:
cardano_comm_df.tail()

,body
created_utc,
2020-05-03 01:13:25,I have no opinion on cardano because I don't k...
2020-05-03 01:06:52,Ok thanks and what do you think about cardano ?
2020-05-01 21:03:44,Which projects do you think have the most pote...
2020-05-01 07:41:28,I have to admit I don't know much about Cardan...
2020-05-01 07:37:30,If a better and more robust solution is the go...


In [139]:
cardano_comm_df.shape

(849, 1)

In [141]:
cardano_comm_df = cardano_comm_df[['body']]
cardano_comm_df.columns = ['cardano_body']
cardano_comm_df.head()

,cardano_body
created_utc,
2020-07-11 13:18:00,cardano has research. at a fundamental level i...
2020-07-11 12:04:11,I recommend watching the Cardano Summit. In 20...
2020-07-11 11:44:16,Only Cardano is. And it pretty much is the Eth...
2020-07-11 08:12:19,"i have bags on both, but VeChain got some dapp..."
2020-07-11 06:30:41,The reason Cardano 'has nothing' is precisely ...


**COMBINE CARDANO DATAFRAMES**

In [142]:
#Combine Posts with Comments and reset index
cardano = pd.concat([cardano_subm_df,cardano_comm_df], axis =0)
cardano = cardano.reset_index(drop=True)
#Drop duplicate copies
cardano.drop_duplicates(keep='first',inplace=True)

cardano.head()

,cardano_body
0,Seem like they feel the pressure from Cardano ...
1,Cardano Reaches Overbought Territory Despite H...
2,Cardano Foundation Warns About Suspicious Acti...
3,Looking to get into altcoins for the first tim...
4,Cardano: ADA keeps climbing! But what is the A...


**SCRAPE ZILLIQA SUBMISSIONS IN CRYPTOCURRENCY SUBREDDIT**

In [143]:
start_epoch=int(dt.datetime(2020, 5, 1).timestamp())

zil_subm = api.search_submissions(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='zilliqa',
                            filter=['title', 'selftext'],
                            limit=None)

In [144]:
zil_subm_df = pd.DataFrame([thing.d_ for thing in zil_subm])

In [145]:
zil_subm_df.tail()

,created,created_utc,selftext,title
31,1.589512e+09,1589497154,,Zilliqa Can Make Everyone’s Rich
32,1.589505e+09,1589490500,,Zilliqa just reached 4 million transactions. 1...
33,1.589166e+09,1589151269,,Platform projects last 30 days correlation mat...
34,1.588957e+09,1588942907,,Zilliqa just launched SocialPay. People can no...
35,1.588592e+09,1588577597,,"Zilliqa has never looked this good, amazing pa..."


In [146]:
zil_subm_df.drop(['created'], axis=1, inplace=True)

In [147]:
zil_subm_df['created_utc'] = pd.to_datetime(zil_subm_df['created_utc'],unit='s')

In [148]:
zil_subm_df.set_index('created_utc', inplace=True)

In [149]:
zil_subm_df.head()

,selftext,title
created_utc,,
2020-07-10 17:06:17,,Binance backed stablecoin BUSD comes to the ZI...
2020-07-10 14:11:11,,Binance-backed BUSD to come to Zilliqa blockchain
2020-07-10 14:09:28,,Binance-backed BUSD to come to Zilliqa blockchain
2020-07-08 00:12:53,,For the zilliqa fans
2020-07-01 18:19:34,[removed],Why i’m bullish on Zilliqa (long read)


In [150]:
zil_subm_df.shape

(36, 2)

In [152]:
#fill in null values with a placeholder
zil_subm_df.fillna('.', inplace = True)
#create column 'body' that combines 'title' with 'selftext'
zil_subm_df['zil_body'] = zil_subm_df['title'] + ' ' + zil_subm_df['selftext']
#subset dataframe of zil posts
zil_subm_df = zil_subm_df[['zil_body']]

zil_subm_df.head()

,zil_body
created_utc,
2020-07-10 17:06:17,Binance backed stablecoin BUSD comes to the ZI...
2020-07-10 14:11:11,Binance-backed BUSD to come to Zilliqa blockch...
2020-07-10 14:09:28,Binance-backed BUSD to come to Zilliqa blockch...
2020-07-08 00:12:53,For the zilliqa fans
2020-07-01 18:19:34,Why i’m bullish on Zilliqa (long read) [removed]


**SCRAPE ZILLIQA COMMENTS IN CRYPTOCURRENCY SUBREDDIT**

In [153]:
# zil_comm = api.search_comments(q='zil', subreddit='CryptoCurrency', filter='body')

# max_response_cache = None
# cache_zil_comm = []

# for c in zil_comm:
#     cache_zil_comm.append(c)

# if False:
#     for c in zil_comm:
#         cache_zil_comm.append(c)

start_epoch=int(dt.datetime(2020, 5, 1).timestamp())

zil_comm = api.search_comments(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='zilliqa',
                            filter=['body'],
                            limit=None)


In [154]:
zil_comm_df = pd.DataFrame([thing.d_ for thing in zil_comm])

In [155]:
zil_comm_df.tail()

,body,created,created_utc
59,Why didn't I put anything into Zilliqa?,1.589377e+09,1589362794
60,Zilliqa,1.588735e+09,1588720923
61,"All except for Zilliqa, which is crushing it r...",1.588591e+09,1588576721
62,Why is zilliqa going up like 7% this hour? Is...,1.588575e+09,1588560910
63,"Hahahahahahaha my man, look at zilliqa goin ...",1.588575e+09,1588560738


In [156]:
zil_comm_df.drop(['created'], axis=1, inplace=True)

In [157]:
zil_comm_df['created_utc'] = pd.to_datetime(zil_comm_df['created_utc'],unit='s')

In [158]:
zil_comm_df.set_index('created_utc', inplace=True)

In [159]:
zil_comm_df.head()

,body
created_utc,
2020-07-10 23:33:33,"&gt;pay a fortune in gas fees, which will only..."
2020-07-09 11:58:22,"Nic projects, although I'm not a fan of XRP.. ..."
2020-07-09 00:50:16,If everyone invests £20-50/ $30-60 into Zilliq...
2020-07-09 00:50:08,If everyone invests £20-50/ $30-60 into Zilliq...
2020-07-09 00:48:06,If everyone invests £20-50/ $30-60 into Zilliq...


In [160]:
zil_comm_df.shape

(64, 1)

In [162]:
zil_comm_df = zil_comm_df[['body']]
zil_comm_df.columns = ['zil_body']
zil_comm_df.head()

,zil_body
created_utc,
2020-07-10 23:33:33,"&gt;pay a fortune in gas fees, which will only..."
2020-07-09 11:58:22,"Nic projects, although I'm not a fan of XRP.. ..."
2020-07-09 00:50:16,If everyone invests £20-50/ $30-60 into Zilliq...
2020-07-09 00:50:08,If everyone invests £20-50/ $30-60 into Zilliq...
2020-07-09 00:48:06,If everyone invests £20-50/ $30-60 into Zilliq...


**COMBINE ZILLIQA DATAFRAMES**

In [163]:
#Combine Posts with Comments and reset index
zil = pd.concat([zil_subm_df,zil_comm_df], axis =0)
zil = zil.reset_index(drop=True)
#Drop duplicate copies
zil.drop_duplicates(keep='first',inplace=True)

zil.head()

,zil_body
0,Binance backed stablecoin BUSD comes to the ZI...
1,Binance-backed BUSD to come to Zilliqa blockch...
3,For the zilliqa fans
4,Why i’m bullish on Zilliqa (long read) [removed]
6,Why I’m bullish on Zilliqa (long read) [removed]


**SCRAPE CHAINLINK SUBMISSIONS IN CRYPTOCURRENCY SUBREDDIT**

In [164]:
start_epoch=int(dt.datetime(2020, 5, 1).timestamp())

link_subm = api.search_submissions(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='chainlink',
                            filter=['title', 'selftext'],
                            limit=None)


In [165]:
link_subm_df = pd.DataFrame([thing.d_ for thing in link_subm])

In [166]:
link_subm_df.tail()

,created,created_utc,selftext,title
72,1.588877e+09,1588862914,,Vulcan Link launches Chainlink Oracle price fe...
73,1.588758e+09,1588743719,,Chainlink: The path to $100 using on-chain ana...
74,1.588540e+09,1588526027,,Chainlink fact sheet
75,1.588471e+09,1588456494,[removed],The baseline protocol and chainlink
76,1.588359e+09,1588344582,[removed],Chainlink Virtual Trivia Night


In [167]:
link_subm_df.drop(['created'], axis=1, inplace=True)

In [168]:
link_subm_df['created_utc'] = pd.to_datetime(link_subm_df['created_utc'],unit='s')

In [169]:
link_subm_df.set_index('created_utc', inplace=True)

In [170]:
link_subm_df.head()

,selftext,title
created_utc,,
2020-07-11 07:09:43,,Anyone else scared of the 5 whales holding 5% ...
2020-07-10 15:22:24,,Invest in Chainlink: Why this Cryptocurrency c...
2020-07-10 08:32:44,\nI'll be a millionaire. I'll prove everyone w...,"Yes, this is it, this is my time, I will be lu..."
2020-07-09 20:21:33,[removed],Chainlink works?
2020-07-09 04:58:49,So I’ve got some money to invest and I’ll also...,Torn - Stellar or Chainlink


In [171]:
link_subm_df.shape

(77, 2)

In [172]:
#fill in null values with a placeholder
link_subm_df.fillna('.', inplace = True)
#create column 'body' that combines 'title' with 'selftext'
link_subm_df['link_body'] = link_subm_df['title'] + ' ' + link_subm_df['selftext']
#subset dataframe of zil posts
link_subm_df = link_subm_df[['link_body']]

link_subm_df.head()

,link_body
created_utc,
2020-07-11 07:09:43,Anyone else scared of the 5 whales holding 5% ...
2020-07-10 15:22:24,Invest in Chainlink: Why this Cryptocurrency c...
2020-07-10 08:32:44,"Yes, this is it, this is my time, I will be lu..."
2020-07-09 20:21:33,Chainlink works? [removed]
2020-07-09 04:58:49,Torn - Stellar or Chainlink So I’ve got some m...


**SCRAPE zil COMMENTS IN CRYPTOCURRENCY SUBREDDIT**

In [173]:
start_epoch=int(dt.datetime(2020, 5, 1).timestamp())

link_comm = api.search_comments(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='chainlink',
                            filter=['body'],
                            limit=None)

In [174]:
link_comm_df = pd.DataFrame([thing.d_ for thing in link_comm])

In [175]:
link_comm_df.tail()

,body,created,created_utc
382,Read this article: https://blockonomi.com/tezo...,1.588338e+09,1588323142
383,"Why are coins like NANO, VIDT and IOTA heavily...",1.588334e+09,1588319759
384,Network effects will cause likely only one pro...,1.588332e+09,1588317930
385,I think the vital thing to understand is that ...,1.588325e+09,1588310450
386,some rich fuck paid someone to hype up link sa...,1.588324e+09,1588309999


In [176]:
link_comm_df.drop(['created'], axis=1, inplace=True)

In [177]:
link_comm_df['created_utc'] = pd.to_datetime(link_comm_df['created_utc'],unit='s')

In [178]:
link_comm_df.set_index('created_utc', inplace=True)

In [179]:
link_comm_df.head()

,body
created_utc,
2020-07-11 07:27:56,Chainlink has done pretty well for the past ye...
2020-07-11 00:43:22,Chainlink can tick all these boxes:\n\n𝗜𝗻𝗳𝗶𝗻𝗶𝘁...
2020-07-10 20:08:54,"&gt;If you're in crypto, all your eggs are in ..."
2020-07-10 13:22:12,"Chainlink is the best play right now, bar none"
2020-07-10 12:56:00,Have you never heard of chainlink??


In [180]:
link_comm_df.shape

(387, 1)

In [181]:
link_comm_df = link_comm_df[['body']]
link_comm_df.columns = ['link_body']
link_comm_df.head()

,link_body
created_utc,
2020-07-11 07:27:56,Chainlink has done pretty well for the past ye...
2020-07-11 00:43:22,Chainlink can tick all these boxes:\n\n𝗜𝗻𝗳𝗶𝗻𝗶𝘁...
2020-07-10 20:08:54,"&gt;If you're in crypto, all your eggs are in ..."
2020-07-10 13:22:12,"Chainlink is the best play right now, bar none"
2020-07-10 12:56:00,Have you never heard of chainlink??


**COMBINE CHAINLINK DATAFRAMES**

In [182]:
#Combine Posts with Comments and reset index
link = pd.concat([link_subm_df,link_comm_df], axis =0)
link = link.reset_index(drop=True)
#Drop duplicate copies
link.drop_duplicates(keep='first',inplace=True)

link.head()

,link_body
0,Anyone else scared of the 5 whales holding 5% ...
1,Invest in Chainlink: Why this Cryptocurrency c...
2,"Yes, this is it, this is my time, I will be lu..."
3,Chainlink works? [removed]
4,Torn - Stellar or Chainlink So I’ve got some m...


**COMBINE ZILLIQA, CARDANO & CHAINLINK DF'S**

In [183]:
#create dataframe of two columns: zil_body & cardano_body
crypto_cc_df = pd.concat([zil,cardano,link],axis=1)
crypto_cc_df.dropna()
crypto_cc_df.reset_index(drop=True,inplace=True)

crypto_cc_df.head()

,zil_body,cardano_body,link_body
0,Binance backed stablecoin BUSD comes to the ZI...,Seem like they feel the pressure from Cardano ...,Anyone else scared of the 5 whales holding 5% ...
1,Binance-backed BUSD to come to Zilliqa blockch...,Cardano Reaches Overbought Territory Despite H...,Invest in Chainlink: Why this Cryptocurrency c...
2,NaN,Cardano Foundation Warns About Suspicious Acti...,"Yes, this is it, this is my time, I will be lu..."
3,For the zilliqa fans,Looking to get into altcoins for the first tim...,Chainlink works? [removed]
4,Why i’m bullish on Zilliqa (long read) [removed],Cardano: ADA keeps climbing! But what is the A...,Torn - Stellar or Chainlink So I’ve got some m...


**ANALYZE SENTIMENT**

In [184]:
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [185]:
zil_sent = {
    "zil_compound": [],
    "zil_pos": [],
    "zil_neu": [],
    "zil_neg": [],
    "zil_sent": [],
}
cardano_sent = {
    "cardano_compound": [],
    "cardano_pos": [],
    "cardano_neu": [],
    "cardano_neg": [],
    "cardano_sent": [],
}
link_sent = {
    "link_compound": [],
    "link_pos": [],
    "link_neu": [],
    "link_neg": [],
    "link_sent": [],
}

# Get sentiment for the text and the title
for index, row in crypto_cc_df.iterrows():
    try:
        # Sentiment scoring with VADER
        zil_sentiment = analyzer.polarity_scores(row["zil_body"])
        zil_sent["zil_compound"].append(zil_sentiment["compound"])
        zil_sent["zil_pos"].append(zil_sentiment["pos"])
        zil_sent["zil_neu"].append(zil_sentiment["neu"])
        zil_sent["zil_neg"].append(zil_sentiment["neg"])
        zil_sent["zil_sent"].append(get_sentiment(zil_sentiment["compound"]))

        cardano_sentiment = analyzer.polarity_scores(row["cardano_body"])
        cardano_sent["cardano_compound"].append(cardano_sentiment["compound"])
        cardano_sent["cardano_pos"].append(cardano_sentiment["pos"])
        cardano_sent["cardano_neu"].append(cardano_sentiment["neu"])
        cardano_sent["cardano_neg"].append(cardano_sentiment["neg"])
        cardano_sent["cardano_sent"].append(get_sentiment(cardano_sentiment["compound"]))
        
        link_sentiment = analyzer.polarity_scores(row["link_body"])
        link_sent["link_compound"].append(link_sentiment["compound"])
        link_sent["link_pos"].append(link_sentiment["pos"])
        link_sent["link_neu"].append(link_sentiment["neu"])
        link_sent["link_neg"].append(link_sentiment["neg"])
        link_sent["link_sent"].append(get_sentiment(link_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
zil_sentiment = pd.DataFrame(zil_sent)
cardano_sentiment = pd.DataFrame(cardano_sent)
link_sentiment = pd.DataFrame(link_sent)
crypto_df = crypto_cc_df.join(zil_sentiment).join(cardano_sentiment).join(link_sentiment)

crypto_df.head()

,zil_body,cardano_body,link_body,zil_compound,zil_pos,zil_neu,zil_neg,zil_sent,cardano_compound,cardano_pos,cardano_neu,cardano_neg,cardano_sent,link_compound,link_pos,link_neu,link_neg,link_sent
0,Binance backed stablecoin BUSD comes to the ZI...,Seem like they feel the pressure from Cardano ...,Anyone else scared of the 5 whales holding 5% ...,0.0258,0.121,0.879,0.0,0.0,0.0772,0.182,0.657,0.161,1.0,-0.6858,0.037,0.775,0.187,-1.0
1,Binance-backed BUSD to come to Zilliqa blockch...,Cardano Reaches Overbought Territory Despite H...,Invest in Chainlink: Why this Cryptocurrency c...,0.0000,0.000,1.000,0.0,0.0,0.0516,0.146,0.854,0.000,1.0,0.0000,0.000,1.000,0.000,0.0
2,NaN,Cardano Foundation Warns About Suspicious Acti...,"Yes, this is it, this is my time, I will be lu...",0.0000,0.000,1.000,0.0,0.0,-0.5544,0.063,0.823,0.114,-1.0,0.0000,0.000,1.000,0.000,0.0
3,For the zilliqa fans,Looking to get into altcoins for the first tim...,Chainlink works? [removed],0.0000,0.000,1.000,0.0,0.0,-0.3382,0.000,0.790,0.210,-1.0,0.7263,0.128,0.812,0.060,1.0
4,Why i’m bullish on Zilliqa (long read) [removed],Cardano: ADA keeps climbing! But what is the A...,Torn - Stellar or Chainlink So I’ve got some m...,0.0000,0.000,1.000,0.0,0.0,0.9986,0.116,0.844,0.040,1.0,0.0000,0.000,1.000,0.000,0.0


In [186]:
crypto_df.hvplot.step(y=['zil_sent', 'cardano_sent', 'link_sent'],xlim=(0, 90))

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [188]:
crypto_df.describe()

,zil_compound,zil_pos,zil_neu,zil_neg,zil_sent,cardano_compound,cardano_pos,cardano_neu,cardano_neg,cardano_sent,link_compound,link_pos,link_neu,link_neg,link_sent
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,87.000000,87.000000,87.000000,87.000000,87.000000
mean,0.371131,0.119239,0.846398,0.034375,0.477273,0.200827,0.093557,0.858830,0.047625,0.352273,0.176614,0.076874,0.890989,0.032149,0.229885
std,0.455016,0.123014,0.136469,0.068711,0.677678,0.425053,0.114405,0.149145,0.108284,0.695466,0.408871,0.106641,0.125795,0.058874,0.676880
min,-0.853900,0.000000,0.468000,0.000000,-1.000000,-0.752100,0.000000,0.470000,0.000000,-1.000000,-0.761700,0.000000,0.549000,0.000000,-1.000000
25%,0.000000,0.000000,0.781750,0.000000,0.000000,0.000000,0.000000,0.785500,0.000000,0.000000,0.000000,0.000000,0.810500,0.000000,0.000000
50%,0.361200,0.107000,0.862000,0.000000,1.000000,0.000000,0.059500,0.858500,0.000000,0.000000,0.000000,0.000000,0.915000,0.000000,0.000000
75%,0.845975,0.162500,1.000000,0.053750,1.000000,0.510600,0.153250,1.000000,0.015500,1.000000,0.463300,0.139000,1.000000,0.046000,1.000000
max,0.999900,0.524000,1.000000,0.492000,1.000000,0.999900,0.412000,1.000000,0.500000,1.000000,0.990700,0.451000,1.000000,0.262000,1.000000
